 # LLM Inference Pipeline — ICD-10 Coding

This notebook runs deterministic (temperature = 0) inference of multiple
large language models for ICD-10 coding of Portuguese obstetric discharge notes.

Models are accessed via API. Clinical data are not included in this repository.

##Importando a tabela e selecionando as colunas de interesse

In [ ]:
import pandas as pd

tabela_auditoria = pd.read_excel("/content/drive/MyDrive/Doutorado - Ricardo- Após a Qualificação/Tabela de Auditoria_Só_as_Colunas_Que Interessam.xlsx")


In [ ]:
tabela_auditoria.columns

In [ ]:
tabela_auditoria = tabela_auditoria[['25. Evolução Alta','21. CID de Alta']]
tabela_auditoria

In [ ]:
tabela_auditoria['21. CID de Alta'] = tabela_auditoria['21. CID de Alta'].str.replace('"', '')
tabela_auditoria

##Rodando a GPT 4o - Mini - Tunado

In [ ]:
import openai
print(openai.__version__)

In [ ]:
import pandas as pd
from typing import List, Literal
from pydantic import BaseModel, ValidationError
import openai
from openai import OpenAI

# Inicialize o cliente OpenAI
client = OpenAI(api_key="APIKEY")

# Modelos Pydantic para validação do JSON de saída
class CIDItem(BaseModel):
    cid: str
    tipo: Literal["principal", "secundário", "terciarios"]

class UltrasoundCIDExtraction(BaseModel):
    cids: List[CIDItem]

def extrair_cid(texto: str) -> List[str]:
    """
    Chama o modelo fine-tuned e retorna a lista de CIDs extraídos.
    """
    if pd.isna(texto) or str(texto).strip() == "":
        return []

    resposta = client.chat.completions.create(
        model="ft:gpt-4o-mini-2024-07-18:personal:experimento-cid:BczZ76m5",
        messages=[
            {
                "role": "system",
                "content": """Você é um médico especializado na determinação de códigos CID-10 (Classificação Internacional de Doenças)
que atua em um hospital de saúde materna e neonatal.
Ao receber uma nota clínica, identifique exclusivamente os códigos CID-10 mais associados àquela nota,
sem qualquer informação ou explicação adicional.
Utilize apenas diretrizes médicas reconhecidas (OMS, CDC, FDA, NICE).

Leia atentamente a nota clínica abaixo e extraia:
- 1 CID “principal” (motivo principal de internação/alta);
- 1 CID “secundário” (complicações ou comorbidades relevantes);
- Vários CIDs “terciarios” (demais condições associadas).

Sua resposta deve ser UM JSON VÁLIDO no formato de exemplo:

{
  "cids": [
    { "cid": "",    "tipo": "principal"  },
    { "cid": "",  "tipo": "secundário" },
    { "cid": "",  "tipo": "terciarios" },
    { "cid": "",  "tipo": "terciarios" }
  ]
}

Nada além desse JSON — sem comentários ou campos extras.
"""
            },
            {"role": "user", "content": texto},
        ],
        response_format={"type": "json_object"},
        temperature=0.0,
    )

    json_str = resposta.choices[0].message.content
    try:
        obj = UltrasoundCIDExtraction.model_validate_json(json_str)
        return [item.cid for item in obj.cids]
    except ValidationError as e:
        print("ValidationError:", e)
        print("Resposta bruta:", json_str)
        return []

tabela_auditoria = tabela_auditoria.copy()
tabela_auditoria["25. Evolução Alta"] = tabela_auditoria["25. Evolução Alta"].fillna("")

# Aplica a função de extração de CID em cada nota clínica
tabela_auditoria["API modelo Tunado"] = tabela_auditoria["25. Evolução Alta"].apply(extrair_cid)

# Exibe as primeiras linhas para conferência
print(tabela_auditoria.head())



In [ ]:
tabela_auditoria["25. Evolução Alta"] = tabela_auditoria["25. Evolução Alta"].fillna("").astype(str)

In [ ]:
tabela_auditoria

##GPT Mini

In [ ]:
import pandas as pd
from typing import List, Literal
from pydantic import BaseModel, ValidationError
import openai
from openai import OpenAI

# Inicialize o cliente OpenAI
client = OpenAI(api_key="APIKEY")

# Modelos Pydantic para validação do JSON de saída
class CIDItem(BaseModel):
    cid: str
    tipo: Literal["principal", "secundário", "terciarios"]

class UltrasoundCIDExtraction(BaseModel):
    cids: List[CIDItem]

def extrair_cid(texto: str) -> List[str]:
    """
    Chama o modelo fine-tuned e retorna a lista de CIDs extraídos.
    """
    if pd.isna(texto) or str(texto).strip() == "":
        return []

    resposta = client.chat.completions.create(
        model="gpt-4o-mini-2024-07-18",
        messages=[
            {
                "role": "system",
                "content": """Você é um médico especializado na determinação de códigos CID-10 (Classificação Internacional de Doenças)
que atua em um hospital de saúde materna e neonatal.
Ao receber uma nota clínica, identifique exclusivamente os códigos CID-10 mais associados àquela nota,
sem qualquer informação ou explicação adicional.
Utilize apenas diretrizes médicas reconhecidas (OMS, CDC, FDA, NICE).

Leia atentamente a nota clínica abaixo e extraia:
- 1 CID “principal” (motivo principal de internação/alta);
- 1 CID “secundário” (complicações ou comorbidades relevantes);
- Vários CIDs “terciarios” (demais condições associadas).

Sua resposta deve ser UM JSON VÁLIDO no formato de exemplo:

{
  "cids": [
    { "cid": "O80",    "tipo": "principal"  },
    { "cid": "O99.8",  "tipo": "secundário" },
    { "cid": "Z39.0",  "tipo": "terciarios" },
    { "cid": "F53.0",  "tipo": "terciarios" }
  ]
}

Nada além desse JSON — sem comentários ou campos extras.
"""
            },
            {"role": "user", "content": texto},
        ],
        response_format={"type": "json_object"},
        temperature=0.0,
    )

    json_str = resposta.choices[0].message.content
    try:
        obj = UltrasoundCIDExtraction.model_validate_json(json_str)
        return [item.cid for item in obj.cids]
    except ValidationError as e:
        print("ValidationError:", e)
        print("Resposta bruta:", json_str)
        return []

tabela_auditoria = tabela_auditoria.copy()
tabela_auditoria["25. Evolução Alta"] = tabela_auditoria["25. Evolução Alta"].fillna("")

# Aplica a função de extração de CID em cada nota clínica
tabela_auditoria["API GPT-Mini"] = tabela_auditoria["25. Evolução Alta"].apply(extrair_cid)

# Exibe as primeiras linhas para conferência
print(tabela_auditoria.head())




In [ ]:
tabela_auditoria["25. Evolução Alta"] = tabela_auditoria["25. Evolução Alta"].fillna("").astype(str)

In [ ]:
tabela_auditoria

##Rodando GPT - 4o

In [ ]:
import pandas as pd
from typing import List, Literal
from pydantic import BaseModel, ValidationError
import openai
from openai import OpenAI

# Inicialize o cliente OpenAI
client = OpenAI(api_key="APIKEY")

# Modelos Pydantic para validação do JSON de saída
class CIDItem(BaseModel):
    cid: str
    tipo: Literal["principal", "secundário", "terciarios"]

class UltrasoundCIDExtraction(BaseModel):
    cids: List[CIDItem]

def extrair_cid(texto: str) -> List[str]:
    """
    Chama o modelo fine-tuned e retorna a lista de CIDs extraídos.
    """
    if pd.isna(texto) or str(texto).strip() == "":
        return []

    resposta = client.chat.completions.create(
        model="gpt-4o-2024-08-06",
        messages=[
            {
                "role": "system",
                "content": """Você é um médico especializado na determinação de códigos CID-10 (Classificação Internacional de Doenças)
que atua em um hospital de saúde materna e neonatal.
Ao receber uma nota clínica, identifique exclusivamente os códigos CID-10 mais associados àquela nota,
sem qualquer informação ou explicação adicional.
Utilize apenas diretrizes médicas reconhecidas (OMS, CDC, FDA, NICE).

Leia atentamente a nota clínica abaixo e extraia:
- 1 CID “principal” (motivo principal de internação/alta);
- 1 CID “secundário” (complicações ou comorbidades relevantes);
- Vários CIDs “terciarios” (demais condições associadas).

Sua resposta deve ser UM JSON VÁLIDO no formato de exemplo:

{
  "cids": [
    { "cid": "O80",    "tipo": "principal"  },
    { "cid": "O99.8",  "tipo": "secundário" },
    { "cid": "Z39.0",  "tipo": "terciarios" },
    { "cid": "F53.0",  "tipo": "terciarios" }
  ]
}

Nada além desse JSON — sem comentários ou campos extras.
"""
            },
            {"role": "user", "content": texto},
        ],
        response_format={"type": "json_object"},
        temperature=0.0,
    )

    json_str = resposta.choices[0].message.content
    try:
        obj = UltrasoundCIDExtraction.model_validate_json(json_str)
        return [item.cid for item in obj.cids]
    except ValidationError as e:
        print("ValidationError:", e)
        print("Resposta bruta:", json_str)
        return []


tabela_auditoria = tabela_auditoria.copy()
tabela_auditoria["25. Evolução Alta"] = (
    tabela_auditoria["25. Evolução Alta"].fillna("")
)

tabela_auditoria["API GPT‑4o"] = (
    tabela_auditoria["25. Evolução Alta"].apply(extrair_cid)
)

print(tabela_auditoria.head())



In [ ]:
tabela_auditoria["25. Evolução Alta"] = tabela_auditoria["25. Evolução Alta"].fillna("").astype(str)

In [ ]:
tabela_auditoria

##Rodando Sabia 3

In [ ]:
import pandas as pd
from typing import List, Literal
from pydantic import BaseModel, ValidationError
import openai
from openai import OpenAI

client = openai.OpenAI(
    api_key="APIKEY",
    base_url="https://chat.maritaca.ai/api",
)
# Modelos Pydantic para validação do JSON de saída
class CIDItem(BaseModel):
    cid: str
    tipo: Literal["principal", "secundário", "terciarios"]

class UltrasoundCIDExtraction(BaseModel):
    cids: List[CIDItem]

def extrair_cid(texto: str) -> List[str]:

    if pd.isna(texto) or str(texto).strip() == "":
        return []

    resposta = client.chat.completions.create(
        model="sabia-3.1",
        messages=[
            {
                "role": "system",
                "content": """Você é um médico especializado na determinação de códigos CID-10 (Classificação Internacional de Doenças)
que atua em um hospital de saúde materna e neonatal.
Ao receber uma nota clínica, identifique exclusivamente os códigos CID-10 mais associados àquela nota,
sem qualquer informação ou explicação adicional.
Utilize apenas diretrizes médicas reconhecidas (OMS, CDC, FDA, NICE).

Leia atentamente a nota clínica abaixo e extraia:
- 1 CID “principal” (motivo principal de internação/alta);
- 1 CID “secundário” (complicações ou comorbidades relevantes);
- Vários CIDs “terciarios” (demais condições associadas).

Sua resposta deve ser UM JSON VÁLIDO no formato de exemplo:

{
  "cids": [
    { "cid": "O80",    "tipo": "principal"  },
    { "cid": "O99.8",  "tipo": "secundário" },
    { "cid": "Z39.0",  "tipo": "terciarios" },
    { "cid": "F53.0",  "tipo": "terciarios" }
  ]
}

Nada além desse JSON — sem comentários ou campos extras.
"""
            },
            {"role": "user", "content": texto},
        ],
        response_format={"type": "json_object"},
        temperature=0.0,
    )

    json_str = resposta.choices[0].message.content
    try:
        obj = UltrasoundCIDExtraction.model_validate_json(json_str)
        return [item.cid for item in obj.cids]
    except ValidationError as e:
        print("ValidationError:", e)
        print("Resposta bruta:", json_str)
        return []


tabela_auditoria = tabela_auditoria.copy()
tabela_auditoria["25. Evolução Alta"] = (
    tabela_auditoria["25. Evolução Alta"].fillna("")
)

tabela_auditoria["API Maritalk"] = (
    tabela_auditoria["25. Evolução Alta"].apply(extrair_cid)
)

print(tabela_auditoria.head())

In [ ]:
tabela_auditoria

##Rodando DeepSeek

In [ ]:
import pandas as pd
import requests
from typing import List
import numpy as np

# Configuração da API
API_KEY = "APIKEY"
API_URL = "https://api.deepseek.com/v1/chat/completions"

def extrair_cid(texto) -> List[str]:
    # Garantir que o valor seja sempre uma string
    texto = str(texto) if not isinstance(texto, str) else texto

    headers = {
        "Authorization": f"Bearer {API_KEY}",
        "Content-Type": "application/json"
    }

    payload = {
        "model": "deepseek-chat",
        "messages": [
            {
                "role": "system",
                "content":
                    """"Você é um médico especializado na determinação de códigos CID-10 (Classificação Internacional de Doenças)
que atua em um hospital de saúde materna e neonatal.
Ao receber uma nota clínica, identifique exclusivamente os códigos CID-10 mais associados àquela nota,
sem qualquer informação ou explicação adicional.
Utilize apenas diretrizes médicas reconhecidas (OMS, CDC, FDA, NICE).

Leia atentamente a nota clínica abaixo e extraia:
- 1 CID “principal” (motivo principal de internação/alta);
- 1 CID “secundário” (complicações ou comorbidades relevantes);
- Vários CIDs “terciarios” (demais condições associadas).

Sua resposta deve ser UM JSON VÁLIDO no formato de exemplo:

{
  "cids": [
    { "cid": "O80",    "tipo": "principal"  },
    { "cid": "O99.8",  "tipo": "secundário" },
    { "cid": "Z39.0",  "tipo": "terciarios" },
    { "cid": "F53.0",  "tipo": "terciarios" }
  ]
}

Nada além desse JSON — sem comentários ou campos extras.
"""
            },
            {
                "role": "user",
                "content": texto
            }
        ],
        "temperature": 0.0
    }

    try:
        response = requests.post(API_URL, json=payload, headers=headers)
        response.raise_for_status()

        # Processamento da resposta
        resposta = response.json()
        conteudo = resposta["choices"][0]["message"]["content"]

        # Extrai códigos (ex.: "A15.0, J18.9" → ["A15.0", "J18.9"])
        return [cid.strip() for cid in conteudo.split(",") if cid.strip()]

    except Exception as e:
        # Convertendo novamente para string, caso necessário, para o slice
        texto_str = str(texto)
        print(f"Erro ao processar '{texto_str[:30]}...': {e}")
        return []


# Substituir valores NaN por string vazia para evitar problemas
tabela_auditoria["25. Evolução Alta"] = tabela_auditoria["25. Evolução Alta"].fillna("")

# Aplicar a função no DataFrame
tabela_auditoria["API Deep-Seek - Sem Prompt"] = tabela_auditoria["25. Evolução Alta"].apply(extrair_cid)

print(tabela_auditoria)


In [ ]:
tabela_auditoria

##API GEMINI

In [ ]:
import pandas as pd
from typing import List, Literal
from pydantic import BaseModel, ValidationError
import google.generativeai as genai
import json
import os


try:
    # Substitua pela sua chave de API do Google AI Studio
    GOOGLE_API_KEY = "APIKEY"
    genai.configure(api_key=GOOGLE_API_KEY)
except Exception as e:
    print(f"Erro ao configurar a API do Gemini: {e}")
    # O script não poderá continuar sem a configuração da API.
    exit()


# --- Modelos Pydantic para Validação (sem alterações) ---
class CIDItem(BaseModel):
    cid: str
    tipo: Literal["principal", "secundário", "terciarios"]

class UltrasoundCIDExtraction(BaseModel):
    cids: List[CIDItem]


# --- Função de Extração Adaptada para o Gemini 1.5 Flash ---
def extrair_cid_gemini(texto: str) -> List[str]:
    if pd.isna(texto) or str(texto).strip() == "":
        return []

    # Configuração do modelo Gemini
    # Usamos o response_mime_type para forçar a saída em JSON.
    generation_config = {
        "temperature": 0.0,
        "response_mime_type": "application/json",
    }

    # O modelo generativo que será usado
    model = genai.GenerativeModel(
        model_name="gemini-1.5-flash",
        generation_config=generation_config
    )

    # O prompt é montado em uma única string, combinando as instruções e o texto do usuário.
    prompt_completo = f"""
Você é um médico especializado na determinação de códigos CID-10 (Classificação Internacional de Doenças) que atua em um hospital de saúde materna e neonatal.
Ao receber uma nota clínica, identifique exclusivamente os códigos CID-10 mais associados àquela nota, sem qualquer informação ou explicação adicional.
Utilize apenas diretrizes médicas reconhecidas (OMS, CDC, FDA, NICE).

Leia atentamente a nota clínica abaixo e extraia:
- 1 CID “principal” (motivo principal de internação/alta);
- 1 CID “secundário” (complicações ou comorbidades relevantes);
- Vários CIDs “terciarios” (demais condições associadas).

Sua resposta deve ser UM JSON VÁLIDO no formato de exemplo:

{{
  "cids": [
    {{ "cid": "O80",    "tipo": "principal"  }},
    {{ "cid": "O99.8",  "tipo": "secundário" }},
    {{ "cid": "Z39.0",  "tipo": "terciarios" }},
    {{ "cid": "F53.0",  "tipo": "terciarios" }}
  ]
}}

Nada além desse JSON — sem comentários, explicações ou campos extras.

---
NOTA CLÍNICA PARA ANÁLISE:
{texto}
---
"""

    try:
        # Chamada para a API do Gemini
        resposta = model.generate_content(prompt_completo)

        # O resultado já vem como um objeto de texto, pronto para ser validado.
        json_str = resposta.text

        # Validação com Pydantic
        obj = UltrasoundCIDExtraction.model_validate_json(json_str)
        return [item.cid for item in obj.cids]

    except ValidationError as e:
        print(f"Erro de validação do Pydantic: {e}")
        print(f"Resposta bruta recebida: {json_str}")
        return []
    except Exception as e:
        print(f"Ocorreu um erro ao chamar a API do Gemini: {e}")
        return []



# Substituir valores NaN por string vazia para evitar problemas
tabela_auditoria["25. Evolução Alta"] = tabela_auditoria["25. Evolução Alta"].fillna("")

# Aplicar a função no DataFrame
tabela_auditoria["API Gemini"] = tabela_auditoria["25. Evolução Alta"].apply(extrair_cid_gemini)

print(tabela_auditoria)

In [ ]:
tabela_auditoria.loc[1].to_frame().T
